In [1]:
import numpy as np
import pandas as pd
import cvxpy as cvx

# Q2
Function, gradient, and hessian definitions

In [2]:
def f(x): 
  x1, x2 = x
  return 100*(x2 -x1**2)**2 + (1 -x1)**2
  
def grad(x):
  x1, x2 = x
  e1 = -400*x1*x2 +400*x1**3 +2*x1 -2
  e2 = 200*x2 -200*x1**2
  return np.array([e1, e2])

def hess(x):  
  x1, x2 = x
  e1 = -400*x2 +1200*x1**2 +2
  e2 = -400*x1
  e3 = -400*x1
  e4 = 200
  return np.array([[e1, e2], [e3, e4]])

x = np.array([1.2, 1.2])
print(f(x))
print(grad(x))
print(hess(x))

5.8
[115.6 -48. ]
[[1250. -480.]
 [-480.  200.]]


Implementation of Newton's Method with Line Search

In [3]:
def Newtons_Method(x0, eps=10**-4, rho=0.95, c=0.5, alpha=1):
  x = [x0]
  alphas = []
  k = 0

  while np.linalg.norm(grad(x[k])) > eps:
    alphas.append(alpha)
    d = -np.linalg.inv(hess(x[k])).dot(grad(x[k]))
    while f(x[k] +alphas[k]*d) > f(x[k]) +c*alphas[k]*grad(x[k]).T.dot(d):
      alphas[k] = rho*alphas[k]

    x.append(x[k] +alphas[k]*d)
    k+=1
  return x, alphas

sol1 = Newtons_Method(np.array([1.2, 1.2]))
sol2 = Newtons_Method(np.array([-1.2, 1]))

In [4]:
def clean_data(solution):
  sol = pd.DataFrame(solution).T
  sol[['x1', 'x2']] = sol[0].to_list()
  sol.drop(0, axis=1, inplace=True)
  sol.rename(columns={1: 'alpha'}, inplace=True)
  return sol

clean_data(sol1)

,alpha,x1,x2
0,1,1.200000,1.200000
1,0.487675,1.195918,1.430204
2,1,1.100691,1.202444
3,0.95,1.064926,1.132788
4,1,1.015811,1.029396
5,1,1.005237,1.010389
6,1,1.000115,1.000203
7,None,1.000001,1.000001


In [5]:
clean_data(sol2)

,alpha,x1,x2
0,1,-1.200000,1.000000
1,0.135276,-1.175281,1.380674
2,1,-0.913063,0.764396
3,0.54036,-0.784301,0.598548
4,1,-0.560904,0.257086
5,0.598737,-0.436085,0.174590
6,1,-0.227181,0.001718
7,0.598737,-0.115400,0.000822
8,1,0.075465,-0.035748
9,0.630249,0.174999,0.020718


# Q4

In [17]:
# Providing Input
c = np.array([16,20,8])
FC_d = np.array([[-1,1,0,0,0,0],
               [0,0,-1,1,0,0],
               [0,0,0,0,-1,1]])

FC_p = np.array([[1,0,0,0,0,-1],
               [0,-1,1,0,0,0],
               [0,0,0,-1,1,0]])

pot_lnk = np.array([[11.6,-11.6,0,0,0,0],
                   [0,5.9,-5.9,0,0,0],
                   [0,0,13.7,-13.7,0,0],
                   [0,0,0,9.8,-9.8,0],
                   [0,0,0,0,5.6,-5.6],
                   [-10.5,0,0,0,0,10.5]])

d_c = np.array([-110,-65,-95])
p_min = np.array([20,20,10])
p_max = np.array([200,150,150])
f_max = np.array([100,110,50,80,60,40])

# Declaring Variables
p = cvx.Variable(3)
f = cvx.Variable(6)
theta = cvx.Variable(6)

# Defining Objective
objective = cvx.Minimize(p@c)

# @ for vector/matrix multiplication
# * for scalar multiplication

# Defining Constraints

constraints =  [FC_p @ f == p,
                FC_d @ f == d_c,
                p >= p_min,
                p <= p_max,
                f >= -f_max,
                f <= f_max,
                pot_lnk @ theta == f
            ]

model = cvx.Problem(objective, constraints)
model.solve()

# Printing Outputs 
print("\nThe optimal value is", round(model.value,2))
print("f values:", f.value)
# Rounded values: 
print("Rounded f values:", [round(i,2) for i in f.value] )
print("Rounded p values:", [round(i,2) for i in p.value] )
print("The dual values for the second set of constraints:", constraints[1].dual_value)


The optimal value is 3304.96
f values: [ 78.12001378 -31.87998622 -11.87998622 -76.87998622  60.
 -35.        ]
Rounded f values: [78.12, -31.88, -11.88, -76.88, 60.0, -35.0]
Rounded p values: [113.12, 20.0, 136.88]
The dual values for the second set of constraints: [14.39887617  9.89520781 17.76886062]
